# **Part 2: Machine Learning**

# CHALLENGE : predict conversions 🏆🏆

# Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
import joblib

# Choose necessary variables

In [ ]:
data = pd.read_csv('C:/Users/elodi/Documents/Documents/PYTHON/Jedha/data_science_full_stack/06_Machine_learning_Supervised/PROJECT_Conversion/Data/conversion_data_train.csv')
data = data[data['age'] <= 100]
# data_sample = data.sample(10000)

In [ ]:
target_variable = 'converted'
Y = data.loc[:, target_variable]

Here we want to check which variables we will further use to test our models. We know from EDA that total_pages_visited seems the most important variable. So we will try to train our model with only this variable, check the f1 score for the train and test, and then add other variables one by one to see if they improve the f1 scores or not. We first add new_user then age, which seemed the most important after the number of pages visited. In each case, we define a new list of features, then a list of numeric_indices and a list of categorical_indices. Then we define the X dataset to work with.

In [ ]:
# Defining features, and features type (numerical/categorical)
features_list = ['total_pages_visited','new_user','age','country','source']
numeric_indices = [0,2]
categorical_indices = [1,3,4]

# Creating several lists of features for our variable study (1 variable, then 2, then 3... then 5)
features_lists = [features_list[:i+1] for i in range(len(features_list))]

# Locate the X dataset in each subset
X_list = [data.loc[:, sublist] for sublist in features_lists]
print("Dataset with only 1 variable: ")
print(X_list[0])
print("")
print("Dataset with 5 variables :")
print(X_list[4])
print("")

# Numeric and categorical indices for each subset
numeric_indices_list = [[i for i in numeric_indices if i in range(len(sublist))] for sublist in features_lists]
categorical_indices_list = [[i for i in categorical_indices if i in range(len(sublist))] for sublist in features_lists]

for i,num,cat in zip(range(5),numeric_indices_list,categorical_indices_list):
    print("Subset with ", i, "variables: Numerical indices: ",num,"; Categorical indices: ",cat)

Here we define the transformers that will be used during preprocessing.

In [ ]:
numeric_transformer = StandardScaler() # for numeric features
categorical_transformer = OneHotEncoder(drop="first") # for categorical features

Now, for each subset (1 or 2 or.. 5 variables), we divide the subset in train and test data, we preprocess them, we run the simplest model, a logistic regression, we make predictions and we assess the model with f1-score.

In [ ]:
# Initialize lists to store results
variables = []
f1_train_scores = []
f1_test_scores = []
precision_scores = []
std_deviations = []

for X, numeric_indices, categorical_indices in zip(X_list, numeric_indices_list, categorical_indices_list):
    
    # Dividing the dataset in train and test (we use stratify as the target variable is not heavenly distributed)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=0, stratify=Y)

    # Applying pipeline on X variables depending on columns
    feature_encoder = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_indices),
            ('num', numeric_transformer, numeric_indices)
        ]
    )
    X_train = feature_encoder.fit_transform(X_train)
    X_test = feature_encoder.transform(X_test)

    # Train the dataset and make predictions
    classifier = LogisticRegression()
    classifier.fit(X_train, Y_train)
    Y_train_pred = classifier.predict(X_train)
    Y_test_pred = classifier.predict(X_test)

    # Calculate scores and precision
    f1_train = np.round(f1_score(Y_train, Y_train_pred), 4)
    f1_test = np.round(f1_score(Y_test, Y_test_pred), 4)
    scores = cross_val_score(classifier, X_train, Y_train, cv=10)
    precision = np.round(scores.mean(), 4)
    std_dev = np.round(scores.std(), 6)

    # Store results in lists
    variables.append(len(numeric_indices) + len(categorical_indices))
    f1_train_scores.append(f1_train)
    f1_test_scores.append(f1_test)
    precision_scores.append(precision)
    std_deviations.append(std_dev)

# Create a DataFrame with the results
results_df = pd.DataFrame({
    'Variables': variables,
    'F1 Score (Train)': f1_train_scores,
    'F1 Score (Test)': f1_test_scores,
    'Precision': precision_scores,
    'Standard Deviation': std_deviations
})

print(results_df)

Every variable added to total_pages_visited is improving the f1-score of the train and test sets. So we confirm that we will use the 5 of them when testing our models later.

# Baseline Logistic Regression

In [ ]:
# Re-defining the dataset with the option chosen: 5 variables
features_list = ['total_pages_visited','new_user','age','country','source']
numeric_indices = [0,2]
categorical_indices = [1,3,4]
X = data.loc[:, features_list]
# I don't need to redefine Y and repreprocess it
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0, stratify=Y)

feature_encoder = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_indices),
        ('num', numeric_transformer, numeric_indices)
    ]
)
X_train = feature_encoder.fit_transform(X_train)
X_test = feature_encoder.transform(X_test) # Don't fit again !!

In [ ]:
model = {
        'name': 'Logistic Regression',
        'model': LogisticRegression(),
        'params': 
            {
            'penalty': ['l1', 'l2'],                                    # Regularization type
            'C': [0.01, 0.1, 1.0, 10.0, 100, 1000, 5000, 8000],         # Inverse of regularization strength
            'solver': ['liblinear', 'lbfgs','newton-cg','sag','saga'],  # Optimization algorithm for smaller datasets
            'max_iter': [100, 200, 500, 1000, 5000],                    # Maximum number of iterations to converge
            'tol': [1e-3, 1e-4, 1e-5],                                  # Tolerance for stopping criteria
            }
        }

In [ ]:
%%time

results = [] # This is where we will save all models with metrics

grid_search = GridSearchCV(model['model'], model['params'], cv=10, scoring='f1', n_jobs=-1)
grid_search.fit(X_train, Y_train)

# Evaluation and storage of performance in a variable "results"
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
train_preds = best_model.predict(X_train)
test_preds = best_model.predict(X_test)
f1_train = np.round(f1_score(Y_train, train_preds), 4)
f1_test = np.round(f1_score(Y_test, test_preds), 4)
# scores = cross_val_score(model, X_train, Y_train, cv=10)
# precision = np.round(scores.mean(), 4)
# std_dev = np.round(scores.std(), 6)
accuracy_train = accuracy_score(Y_train, train_preds)
accuracy_test = accuracy_score(Y_test, test_preds)
recall = recall_score(Y_test, test_preds)
auc_roc = roc_auc_score(Y_test, test_preds)

# Store the best logistic regression model from the gridsearch
results.append({'Model': model['name'],
                'F1 Train': f1_train,
                'F1 Test': f1_test,
                # 'Precision': precision,
                # 'Std Dev': std_dev,
                'Train Accuracy': accuracy_train,
                'Test Accuracy': accuracy_test,
                'Recall': recall,
                'AUC-ROC': auc_roc,
                'Best Model': best_model,
                'Best Params': best_params,
                'Train Preds': train_preds,
                'Test Preds': test_preds
                })

In [ ]:
cv_results = grid_search.cv_results_

grid_search_results = pd.DataFrame(columns=['Model', 'Mean F1 Score', 'Parameters'])

for mean_score, params in zip(cv_results['mean_test_score'], cv_results['params']):
    grid_search_results = grid_search_results.append({
        'Model': model['name'],
        'Mean F1 Score': mean_score,
        'Parameters': params
    }, ignore_index=True)

In [ ]:
sorted_logistic_regressions = grid_search_results.sort_values(by='Mean F1 Score', ascending=False)
pd.set_option('display.max_colwidth', None)
display(sorted_logistic_regressions.head(10))

We notice here that we get smaller f1 scores that we got in the previous part testing the 5 variables (we got 0.7712 for the test). This is explained by the fact that there was no cross validation made before whereas we do a gridsearch with a cv=10 now, which is much more accurate.

# Testing more models

In [ ]:
# Models to be tested

models = [
    {
        'name': 'SVM Linear',
        'model': SVC(kernel='linear', probability=True),
        'params': {} 
    },
    {
        'name': 'SVM RBF',
        'model': SVC(kernel='rbf', probability=True),
        'params': {}
    },
    {
        'name': 'SVM Poly (3)',
        'model': SVC(kernel='poly', degree=3, probability=True),
        'params': {}
    },
    {
        'name': 'Decision Tree Entropy',
        'model': DecisionTreeClassifier(criterion="entropy"),
        'params': {
            'max_depth': [4, 6, 8, 10],
            'min_samples_leaf': [1, 2, 5],
            'min_samples_split': [2, 4, 8]
        }
    },
    {
        'name': 'Decision Tree Gini',
        'model': DecisionTreeClassifier(criterion="gini"),
        'params': {
            'max_depth': [4, 6, 8, 10],
            'min_samples_leaf': [1, 2, 5],
            'min_samples_split': [2, 4, 8]
        }
    },
    {
        'name': 'Random Forest',
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [10, 50, 100, 200],
            'max_depth': [1, 2, 4, 8],
            'min_samples_leaf': [1, 2, 5],
            'min_samples_split': [2, 4, 8]
        }
    },
    {
        'name': 'XGBoost',
        'model': XGBClassifier(),
        'params': {
            'n_estimators': [100, 150],  # Reduced from [100, 150, 300]
            'learning_rate': [0.1, 0.01],  # Reduced from [0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
            'max_depth': [4, 5],  # Reduced from [4, 5, 6, 7, 10]
            'min_child_weight': [1],  # Only one value
            # 'gamma': [0, 0.1, 0.2]
        }
    },
    {
        'name': 'CatBoost',
        'model': CatBoostClassifier(),
        'params': {
            'n_estimators': [1000],  # Reduced from [1000, 5000]
            'learning_rate': [0.1],  # Reduced from [0.01, 0.05, 0.1, 0.5, 1]
            'depth': [6],  # Only one value
        }
    },
]


In [ ]:
%%time

# Iterate over models, doing a gridsearch for choosing the best params for each model. 
for model in models:
    model_name = model['name']
    model_type = model['model']
    params = model['params']
    grid_search = GridSearchCV(model_type, params, cv=5, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, Y_train)

    # Evaluation and storage of performance in a variable "results"
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    train_preds = best_model.predict(X_train)
    test_preds = best_model.predict(X_test)
    f1_train = np.round(f1_score(Y_train, train_preds), 4)
    f1_test = np.round(f1_score(Y_test, test_preds), 4)
    # scores = cross_val_score(model, X_train, Y_train, cv=10)
    # precision = np.round(scores.mean(), 4)
    # std_dev = np.round(scores.std(), 6)
    accuracy_train = accuracy_score(Y_train, train_preds)
    accuracy_test = accuracy_score(Y_test, test_preds)
    recall = recall_score(Y_test, test_preds)
    auc_roc = roc_auc_score(Y_test, test_preds)

    results.append({'Model': model_name,
                    'F1 Train': f1_train,
                    'F1 Test': f1_test,
                    # 'Precision': precision,
                    # 'Std Dev': std_dev,
                    'Train Accuracy': accuracy_train,
                    'Test Accuracy': accuracy_test,
                    'Recall': recall,
                    'AUC-ROC': auc_roc,
                    'Best Model': best_model,
                    'Best Params': best_params,
                    'Train Preds': train_preds,
                    'Test Preds': test_preds
                    })

In [ ]:
# Results storage in a DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.iloc[:,:7]
sorted_results_df = results_df.sort_values(by='F1 Test', ascending=False)
print(sorted_results_df)

The best model is still the logistic regression.

# Ensemble learning methods

We already tuned the logistic regression with the gridsearch. Now, we can try to improve the f1-score using boosting and bagging ensemble learning methods. We will also try voting and stacking ensemble methods, which can mix several kinds of models. For stacking the top3 models from above (logistic regression, xgboost and SVM linear), the time exceeded more than a day, so we decided to try stacking several types of logistic regressions instead, by using the gridsearch results made previously. 

In [ ]:
# Retrieve the top3 models tested
top3_models = sorted_results_df['Model'].head(3).tolist()
print("Top 3 models: ",top3_models)
top3_result = [result for result in results if result['Model'] in top3_models]

# Retrieve interesting models of logistic regressions
logistic_regression_top1 = {'C': 8000, 'max_iter': 200, 'penalty': 'l1', 'solver': 'saga', 'tol': 0.001}
logistic_regression_top2 = {'C': 10.0, 'max_iter': 200, 'penalty': 'l2', 'solver': 'saga', 'tol': 0.001}
logistic_regression_top3 = {'C': 1000, 'max_iter': 5000, 'penalty': 'l2', 'solver': 'saga', 'tol': 0.001}
logistic_regression_top4 = 	{'C': 10.0, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear', 'tol': 0.001}

In [ ]:
ensemble_methods = [
    {'name': 'Voting_top3models', 'model': VotingClassifier, 'params': {'estimators': [(model['Model'], model['Best Model']) for model in top3_result]}},
    {'name': 'Bagging_20x_logistic_regression_top1', 'model': BaggingClassifier, 'params': {'base_estimator': LogisticRegression(**logistic_regression_top1), 'n_estimators': 20}},
    {'name': 'Bagging_100x_logistic_regression_top1', 'model': BaggingClassifier, 'params': {'base_estimator': LogisticRegression(**logistic_regression_top1), 'n_estimators': 100}},
    {'name': 'Adaboosting_20x_logistic_regression_top1', 'model': AdaBoostClassifier, 'params': {'base_estimator': LogisticRegression(**logistic_regression_top1), 'n_estimators': 20}},
    {'name': 'Adaboosting_100x_logistic_regression_top1', 'model': AdaBoostClassifier, 'params': {'base_estimator': LogisticRegression(**logistic_regression_top1), 'n_estimators': 100}},
    {'name': 'Stacking_logistic_regressions', 'model': StackingClassifier, 'params': {'estimators': [
        ('Logistic_Regression_Top1', LogisticRegression(**logistic_regression_top1)),
        ('Logistic_Regression_Top2', LogisticRegression(**logistic_regression_top2)),
        ('Logistic_Regression_Top3', LogisticRegression(**logistic_regression_top3)),
        ('Logistic_Regression_Top4', LogisticRegression(**logistic_regression_top4))
    ]}},
]

In [ ]:
%%time

for ensemble in ensemble_methods:
    ensemble_name = ensemble['name']
    ensemble_type = ensemble['model']
    params = ensemble.get('params', {})
    
    ensemble_full_model = ensemble_type(**params)
    ensemble_full_model.fit(X_train, Y_train)
    
    train_preds = ensemble_full_model.predict(X_train)
    test_preds = ensemble_full_model.predict(X_test)
    f1_train = np.round(f1_score(Y_train, train_preds), 4)
    f1_test = np.round(f1_score(Y_test, test_preds), 4)
    accuracy_train = accuracy_score(Y_train, train_preds)
    accuracy_test = accuracy_score(Y_test, test_preds)
    recall = recall_score(Y_test, test_preds)
    auc_roc = roc_auc_score(Y_test, test_preds)
    
    results.append({
        'Model': ensemble_name, 
        'F1 Train': f1_train,
        'F1 Test': f1_test,
        # 'Precision': precision,
        # 'Std Dev': std_dev,
        'Train Accuracy': accuracy_train,
        'Test Accuracy': accuracy_test,
        'Recall': recall,
        'AUC-ROC': auc_roc,
        'Best Model': ensemble_full_model,
        'Best Params': params,
        'Train Preds': train_preds,
        'Test Preds': test_preds
        })

In [ ]:
# Results storage in a DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.iloc[:,:7]
sorted_results_df = results_df.sort_values(by='F1 Test', ascending=False)
print(sorted_results_df)

In [ ]:
selected_model_name = "Bagging_100x_logistic_regression_top1"

# Find the selected model in the results list, retrieve predictions
selected_model = next((d for d in results if d["Model"] == selected_model_name), None)
Y_train_pred_selected = selected_model["Train Preds"]
Y_test_pred_selected = selected_model["Test Preds"]

print("Confusion matrix on train set:")
print(confusion_matrix(Y_train, Y_train_pred_selected))
print("Confusion matrix on test set:")
print(confusion_matrix(Y_test, Y_test_pred_selected))

# Add the test_dataset and make predictions

We concatenate our train and test set (from the first dataset: train_dataset) to train the model on a larger set of data.

In [ ]:
X = np.append(X_train,X_test,axis=0)
Y = np.append(Y_train,Y_test)

Then, we retrieve the best models from the whole project.

In [ ]:
top5_models_names = sorted_results_df['Model'].head(5).tolist()
final_results = []

Now, we fit the models on the new larger dataset, retrieve f1 score and other useful metrics.Finally, we take the file "conversion_data_test" that we are asked to predict, preprocess it the same way we did with the train_dataset, then we can apply our model on it and predict the labels. We save all models and predictions in the folder "final_models".

In [ ]:
%%time

for model_name in top5_models_names:
    # Retrieve the model information in the results list
    model_info = next(model for model in results if model['Model'] == model_name)
    best_model = model_info['Best Model']

    # Fit the best model on the new dataset (X, Y)
    best_model.fit(X, Y)

    # Make predictions on the new dataset
    Y_pred = best_model.predict(X)

    # Calculate evaluation metrics on the new dataset and store them
    f1_final = np.round(f1_score(Y, Y_pred), 4)
    accuracy_final = accuracy_score(Y, Y_pred)
    recall_final = recall_score(Y, Y_pred)
    auc_roc_final = roc_auc_score(Y, Y_pred)
    final_results.append({
        'Model': model_name,
        'F1 Score': f1_final,
        'Accuracy': accuracy_final,
        'Recall': recall_final,
        'AUC-ROC': auc_roc_final
    })

    # Saving model
    joblib.dump(best_model, f'final_models/{model_name}.joblib')

    # Predict the the file "conversion_data_test"
    data_without_labels = pd.read_csv('C:/Users/elodi/Documents/Documents/PYTHON/Jedha/data_science_full_stack/06_Machine_learning_Supervised/PROJECT_Conversion/Data/conversion_data_test.csv') # read file
    X_without_labels = data_without_labels.loc[:, features_list].values
    X_without_labels = feature_encoder.transform(X_without_labels) # preprocessing
    data = {'converted': best_model.predict(X_without_labels)} # predict
    Y_predictions = pd.DataFrame(columns=['converted'], data=data) # one column "converted", no index
    prediction_filename = f'C:/Users/elodi/Documents/Documents/PYTHON/Jedha/data_science_full_stack/06_Machine_learning_Supervised/PROJECT_Conversion/Predictions/{model_name}_predictions.csv'
    Y_predictions.to_csv(prediction_filename, index=False) # save into csv file